![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/colab/Training/binary_text_classification/NLU_training_sentiment_classifier_demo_reddit.ipynb)


# Training a Sentiment Analysis Classifier with NLU 
## 2 class Reddit comment sentiment classifier training
With the [SentimentDL model](https://nlp.johnsnowlabs.com/docs/en/annotators#sentimentdl-multi-class-sentiment-analysis-annotator)  from Spark NLP you can achieve State Of the Art results on any multi class text classification problem 

This notebook showcases the following features : 

- How to train the deep learning classifier
- How to store a pipeline to disk
- How to load the pipeline from disk (Enables NLU offline mode)



# 1. Install Java 8 and NLU

In [ ]:
import os
from sklearn.metrics import classification_report
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! pip install  pyspark==2.4.7 
! pip install nlu > /dev/null    



import nlu

# 2. Download Reddit  Sentiment dataset 
https://www.kaggle.com/cosmos98/twitter-and-reddit-sentimental-analysis-dataset
#Context

This is was a Dataset Created as a part of the university Project On Sentimental Analysis On Multi-Source Social Media Platforms using PySpark.

In [ ]:
! wget http://ckl-it.de/wp-content/uploads/2021/01/Reddit_Data.csv


--2021-01-16 09:08:00--  http://ckl-it.de/wp-content/uploads/2021/01/Reddit_Data.csv
Resolving ckl-it.de (ckl-it.de)... 217.160.0.108, 2001:8d8:100f:f000::209
Connecting to ckl-it.de (ckl-it.de)|217.160.0.108|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153265 (150K) [text/csv]
Saving to: ‘Reddit_Data.csv’

Reddit_Data.csv     100%[===================>] 149.67K   402KB/s    in 0.4s    

2021-01-16 09:08:01 (402 KB/s) - ‘Reddit_Data.csv’ saved [153265/153265]



In [ ]:
import pandas as pd
train_path = '/content/Reddit_Data.csv'

train_df = pd.read_csv(train_path)
# the text data to use for classification should be in a column named 'text'
columns=['text','y']
train_df = train_df[columns]
train_df

,text,y
0,its true they had cut the power what douchebag...,positive
1,fuck giroud better finishing like this month,positive
2,looks shit now but still proud made,positive
3,pelor the burning hate the best evil god,negative
4,can ask what you with something this powerful,positive
...,...,...
595,bangali desh bechne main sabse aage,positive
596,national media channels were gaged not cover t...,positive
597,been following these threads from the beginni...,negative
598,pretty sure this sarcasm satire the news 1500...,positive


# 3. Train Deep Learning Classifier using nlu.load('train.sentiment')

You dataset label column should be named 'y' and the feature column with text data should be named 'text'

In [ ]:
import nlu 
# load a trainable pipeline by specifying the train. prefix  and fit it on a datset with label and text columns
# by default the Universal Sentence Encoder (USE) Sentence embeddings are used for generation
trainable_pipe = nlu.load('train.sentiment')
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50])

# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50],output_level='document')
#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
              precision    recall  f1-score   support

    negative       0.67      1.00      0.80        24
     neutral       0.00      0.00      0.00         0
    positive       1.00      0.23      0.38        26

    accuracy                           0.60        50
   macro avg       0.56      0.41      0.39        50
weighted avg       0.84      0.60      0.58        50



,document,y,text,sentiment,default_name_embeddings,sentiment_confidence
origin_index,,,,,,
0,its true they had cut the power what douchebag...,positive,its true they had cut the power what douchebag...,negative,"[0.033111296594142914, 0.053994592279195786, -...",0.632922
1,fuck giroud better finishing like this month,positive,fuck giroud better finishing like this month,neutral,"[0.0678204670548439, 0.01411951333284378, -0.0...",0.558096
2,looks shit now but still proud made,positive,looks shit now but still proud made,neutral,"[0.03247416764497757, -0.09844464808702469, -0...",0.594104
3,pelor the burning hate the best evil god,negative,pelor the burning hate the best evil god,negative,"[0.04032062739133835, 0.07666623592376709, -0....",0.699286
4,can ask what you with something this powerful,positive,can ask what you with something this powerful,positive,"[0.015518003143370152, -0.05116305500268936, -...",0.615222
5,aapâ shazia ilmi from puram constituency lag...,negative,aapâ shazia ilmi from puram constituency lag...,negative,"[0.02478150464594364, -0.06508146971464157, -0...",0.751383
6,fuck yeah,negative,fuck yeah,negative,"[0.04602408409118652, -0.025047995150089264, -...",0.663185
7,honestly really surprised alice ranked that lo...,positive,honestly really surprised alice ranked that lo...,positive,"[-0.035716041922569275, -0.04127982258796692, ...",0.605483
8,didn care about politics before now hate,negative,didn care about politics before now hate,negative,"[-0.006816444452852011, 0.06221264973282814, -...",0.701191


# Test the fitted pipe on new example

In [ ]:
fitted_pipe.predict("Indian prime minister was assinated!")

,document,sentiment,default_name_embeddings,sentiment_confidence
origin_index,,,,
0,Bitcoin is going to the moon!,neutral,"[0.06468033790588379, -0.040837567299604416, -...",0.524234


## Configure pipe training parameters

In [ ]:
trainable_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['sentiment_dl'] has settable params:
pipe['sentiment_dl'].setMaxEpochs(2)                 | Info: Maximum number of epochs to train | Currently set to : 2
pipe['sentiment_dl'].setLr(0.005)                    | Info: Learning Rate | Currently set to : 0.005
pipe['sentiment_dl'].setBatchSize(64)                | Info: Batch size | Currently set to : 64
pipe['sentiment_dl'].setDropout(0.5)                 | Info: Dropout coefficient | Currently set to : 0.5
pipe['sentiment_dl'].setEnableOutputLogs(True)       | Info: Whether to use stdout in addition to Spark logs. | Currently set to : True
pipe['sentiment_dl'].setThreshold(0.6)               | Info: The minimum threshold for the final result otheriwse it will be neutral | Currently set to : 0.6
pipe['sentiment_dl'].setThresholdLabel('neutral')    | Info: In case the score is less than threshold, what should be the label. Default i

## Retrain with new parameters

In [ ]:
# Train longer!
trainable_pipe['sentiment_dl'].setMaxEpochs(5)  
fitted_pipe = trainable_pipe.fit(train_df.iloc[:50])
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df.iloc[:50],output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

preds

              precision    recall  f1-score   support

    negative       1.00      0.83      0.91        24
     neutral       0.00      0.00      0.00         0
    positive       1.00      1.00      1.00        26

    accuracy                           0.92        50
   macro avg       0.67      0.61      0.64        50
weighted avg       1.00      0.92      0.96        50



,document,y,text,sentiment,default_name_embeddings,sentiment_confidence
origin_index,,,,,,
0,its true they had cut the power what douchebag...,positive,its true they had cut the power what douchebag...,positive,"[0.033111296594142914, 0.053994592279195786, -...",0.761194
1,fuck giroud better finishing like this month,positive,fuck giroud better finishing like this month,positive,"[0.0678204670548439, 0.01411951333284378, -0.0...",0.938677
2,looks shit now but still proud made,positive,looks shit now but still proud made,positive,"[0.03247416764497757, -0.09844464808702469, -0...",0.954937
3,pelor the burning hate the best evil god,negative,pelor the burning hate the best evil god,negative,"[0.04032062739133835, 0.07666623592376709, -0....",0.810980
4,can ask what you with something this powerful,positive,can ask what you with something this powerful,positive,"[0.015518003143370152, -0.05116305500268936, -...",0.956043
5,aapâ shazia ilmi from puram constituency lag...,negative,aapâ shazia ilmi from puram constituency lag...,negative,"[0.02478150464594364, -0.06508146971464157, -0...",0.708917
6,fuck yeah,negative,fuck yeah,negative,"[0.04602408409118652, -0.025047995150089264, -...",0.731940
7,honestly really surprised alice ranked that lo...,positive,honestly really surprised alice ranked that lo...,positive,"[-0.035716041922569275, -0.04127982258796692, ...",0.966494
8,didn care about politics before now hate,negative,didn care about politics before now hate,negative,"[-0.006816444452852011, 0.06221264973282814, -...",0.672320


# Try training with different Embeddings

In [ ]:
# We can use nlu.print_components(action='embed_sentence') to see every possibler sentence embedding we could use. Lets use bert!
nlu.print_components(action='embed_sentence')

For language <en> NLU provides the following Models : 
nlu.load('en.embed_sentence') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.tfhub_use') returns Spark NLP model tfhub_use
nlu.load('en.embed_sentence.use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.tfhub_use.lg') returns Spark NLP model tfhub_use_lg
nlu.load('en.embed_sentence.albert') returns Spark NLP model albert_base_uncased
nlu.load('en.embed_sentence.electra') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_small_uncased') returns Spark NLP model sent_electra_small_uncased
nlu.load('en.embed_sentence.electra_base_uncased') returns Spark NLP model sent_electra_base_uncased
nlu.load('en.embed_sentence.electra_large_uncased') returns Spark NLP model sent_electra_large_uncased
nlu.load('en.embed_sentence.bert') returns Spark NLP model sent_bert_base_uncased
nlu.load('en.embed_sentenc

In [ ]:
trainable_pipe = nlu.load('en.embed_sentence.small_bert_L12_768 train.sentiment')
# We need to train longer and user smaller LR for NON-USE based sentence embeddings usually
# We could tune the hyperparameters further with hyperparameter tuning methods like gridsearch
# Also longer training gives more accuracy
trainable_pipe['sentiment_dl'].setMaxEpochs(70)  
trainable_pipe['sentiment_dl'].setLr(0.0005) 
fitted_pipe = trainable_pipe.fit(train_df)
# predict with the trainable pipeline on dataset and get predictions
preds = fitted_pipe.predict(train_df,output_level='document')

#sentence detector that is part of the pipe generates sone NaNs. lets drop them first
preds.dropna(inplace=True)
print(classification_report(preds['y'], preds['sentiment']))

#preds

sent_small_bert_L12_768 download started this may take some time.
Approximate size to download 392.9 MB
[OK!]
              precision    recall  f1-score   support

    negative       0.84      0.77      0.80       300
     neutral       0.00      0.00      0.00         0
    positive       0.91      0.72      0.80       300

    accuracy                           0.74       600
   macro avg       0.59      0.50      0.54       600
weighted avg       0.88      0.74      0.80       600



# 5. Lets save the model

In [ ]:
stored_model_path = './models/classifier_dl_trained' 
fitted_pipe.save(stored_model_path)

Stored model in ./models/classifier_dl_trained


# 6. Lets load the model from HDD.
This makes Offlien NLU usage possible!   
You need to call nlu.load(path=path_to_the_pipe) to load a model/pipeline from disk.

In [ ]:
hdd_pipe = nlu.load(path=stored_model_path)

preds = hdd_pipe.predict('Indian prime minister was assinated')
preds

Fitting on empty Dataframe, could not infer correct training method!


,en_embed_sentence_small_bert_L12_768_embeddings,document,sentiment,sentiment_confidence
origin_index,,,,
0,"[0.15737222135066986, 0.2598555386066437, 0.85...",Tesla plans to invest 10M into the ML sector,positive,0.638827


In [ ]:
hdd_pipe.print_info()

The following parameters are configurable for this NLU pipeline (You can copy paste the examples) :
>>> pipe['document_assembler'] has settable params:
pipe['document_assembler'].setCleanupMode('shrink')            | Info: possible values: disabled, inplace, inplace_full, shrink, shrink_full, each, each_full, delete_full | Currently set to : shrink
>>> pipe['sentence_detector'] has settable params:
pipe['sentence_detector'].setCustomBounds([])                  | Info: characters used to explicitly mark sentence bounds | Currently set to : []
pipe['sentence_detector'].setDetectLists(True)                 | Info: whether detect lists during sentence detection | Currently set to : True
pipe['sentence_detector'].setExplodeSentences(False)           | Info: whether to explode each sentence into a different row, for better parallelization. Defaults to false. | Currently set to : False
pipe['sentence_detector'].setMaxLength(99999)                  | Info: Set the maximum allowed length for ea